# Laboratorio 4 - Regresión y Clustering

## Paula Hípola Gómez, José Ignacio Navas Sanz y Belén Ortega Pérez

## Entregable 2 - Características de los Inmuebles

In [ ]:
%config IPCompleter.greedy=True
%reset
#Importamos las librerías necesarias
#Numpy
import numpy as np
#Pandas
import pandas as pd
#Scikit-Learn
import sklearn as sk
#Matplotlib
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from sklearn import metrics
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

In [ ]:
#Se utiliza para centrar imágenes
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

In [ ]:
#Importamos el CSV
dataset = pd.read_csv('casas.csv')
#Guardamos una copia del dataset original
datasetOriginal = dataset
#Mostramos los datos
dataset.head()

In [ ]:
dataset.describe()

El atributo id no es útil en el estudio del precio de las casas ya que para nosotros significa el orden en el que los inmuebles se han añadido al conjunto de datos. Es por esto que procedemos a eliminarlo. Del mismo modo, el precio de las distintas casas es nuestra variable objetivo por lo que lo separamos del dataset.  Finalmente, obtendremos los 79 atributos que describen las casas de este conjunto de datos.

In [ ]:
#Creamos el array con el precio de cada registro.
precioDeVenta = dataset["SalePrice"].values
#Eliminamos los atributos ID y SalePrice
dataset = dataset.drop(columns=['Id', 'SalePrice'])
dataset.head(10)

In [ ]:
#Miramos cuántos datos duplicados hay
numDuplicados = dataset.duplicated().sum()
print('Hay',numDuplicados,'valores duplicados')

In [ ]:
#Miramos cuántos datos nulos hay
columnasNulas = dataset.isnull().sum()
porcentajes = []
valores = []

#Calculamos el porcentaje de nulos
for col in columnasNulas:
    porcent = (col/dataset.shape[0])*100
    porcentajes.append(porcent)
    
dataFrameNulos = {'Nulos': columnasNulas, 'Porcentaje': porcentajes}
dataFrameNulos = pd.DataFrame(dataFrameNulos)
dataFrameNulos = dataFrameNulos.loc[dataFrameNulos['Nulos'] > 0]
dataFrameNulos

En nuestra opinión, para calcular el precio de una casa, son importantes todos los atributos ya que cada valor que tome el inmueble en cada uno de ellos será el que diferencie el precio. Es por esto que hemos decidido cambiar el valor NaN de nuestro dataset por valores que no sean nulos para un mejor análisis del precio.
Además, hemos detectado que hay ciertas columnas que el valor NA implica NO. No es que sea un valor desconocido, es que las casas no tienen esas propiedades.

In [ ]:
#En las columnas no numéricas, cambiamos los nulos por el valor 'No'
columnasNoNumericas = dataset.dtypes[dataset.dtypes == 'object']
columnasNoNumericas.index

#Hemos detectado que hay ciertas columnas que el NA implica NO por lo que pondremos ese valor realmente, 
#no es que sea desconocido, es que no tienen esa propiedad las casas

columnasNAesNo = ['Alley','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','FireplaceQu',
                  'GarageType','GarageFinish','GarageQual','GarageCond','PoolQC','Fence','MiscFeature',]

for col in columnasNAesNo:
    dataset[col] = dataset[col].fillna('No')

In [ ]:
dataset

In [ ]:
#Actualizamos las columnas nulas
#Volvemos a comprobar cuántos valores nulos hay
columnasNulas = dataset.isnull().sum()
porcentajes = []
valores = []

#Calculamos el porcentaje de nulos
for col in columnasNulas:
    porcent = (col/dataset.shape[0])*100
    porcentajes.append(porcent)
    
dataFrameNulos = {'Nulos': columnasNulas, 'Porcentaje': porcentajes}
dataFrameNulos = pd.DataFrame(dataFrameNulos)
dataFrameNulos = dataFrameNulos.loc[dataFrameNulos['Nulos'] > 0]
dataFrameNulos

In [ ]:
#En las columnas no numéricas, cambiamos los nulos por el valor 'Desconocido'
columnasNoNumericas = dataset.dtypes[dataset.dtypes == 'object']

for col in columnasNoNumericas.index:
    if(col in dataFrameNulos.index):
        dataset[col] = dataset[col].fillna('Desconocido')

In [ ]:
#Actualizamos las columnas nulas
#Volvemos a comprobar cuántos valores nulos hay
columnasNulas = dataset.isnull().sum()
porcentajes = []
valores = []

#Calculamos el porcentaje de nulos
for col in columnasNulas:
    porcent = (col/dataset.shape[0])*100
    porcentajes.append(porcent)
    
dataFrameNulos = {'Nulos': columnasNulas, 'Porcentaje': porcentajes}
dataFrameNulos = pd.DataFrame(dataFrameNulos)
dataFrameNulos = dataFrameNulos.loc[dataFrameNulos['Nulos'] > 0]
dataFrameNulos

La columna GarageYrBlt con 81 nulos es resultante de casas sin garaje. Cambiaremos el valor nulo por un 0 ya que el atributo es de tipo <i>float64</i>